#### TODO：尝试拆分句子为短句

In [1]:
import fasttext
import os
import json
import re
from utils import fixText, resetLabelLv2, resetLabelLv3
from model import MyFastText

TEMP_DIR = './tmp/'

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

def split_text(text):
    splited_text = re.split(r'\n+|\r+|\r\n|<br>', text)
    while '' in splited_text:
        splited_text.remove('')
    return splited_text

config = getConfig()

In [2]:
# 读入数据集的json文件，处理成fasttext接口使用的"文本__label__标签"形式，以txt文件存储
def readDataSet(config):
    set1 = []
    with open(config['data_path'] + config['dataset_label_1_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set1.append('__label__' + data_set[id]['label_level_1'] + ' ' + fixText(text))
    set2 = []
    with open(config['data_path'] + config['dataset_label_2_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set2.append('__label__' + data_set[id]['label_level_2'] + ' ' + fixText(text))
    set3 = []
    with open(config['data_path'] + config['dataset_label_3_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            splited_text = split_text(text)
            for text in splited_text:
                set3.append('__label__' + data_set[id]['label_level_3'] + ' ' + fixText(text))
        
    try:
        os.mkdir(TEMP_DIR)
    except:
        pass
    with open(TEMP_DIR + 'set1.txt', 'w') as f:
        print('训练集1拆分后共{}条文本'.format(len(set1)))
        for l in set1:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set2.txt', 'w') as f:
        print('训练集2拆分后共{}条文本'.format(len(set2)))
        for l in set2:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set3.txt', 'w') as f:
        print('训练集3拆分后共{}条文本'.format(len(set2)))
        for l in set3:
            f.write(l + '\n')

readDataSet(config)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


训练集2拆分后共1285245条文本


In [3]:
model_label_1 = MyFastText()
model_label_1.train(
    input = TEMP_DIR + 'set1.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch'],
)
model_label_1.save(config['model_path'] + config['model_label_1_name'])

Read 2M words
Number of words:  61624
Number of labels: 85
Progress: 100.0% words/sec/thread:  193551 lr:  0.000000 avg.loss:  1.508976 ETA:   0h 0m 0s


In [3]:
model_label_2 = MyFastText()
model_label_2.train(
    input = TEMP_DIR + 'set2.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)
model_label_2.save(config['model_path'] + config['model_label_2_name'])

Read 20M words
Number of words:  233142
Number of labels: 453
Progress:   1.1% words/sec/thread:   37475 lr:  0.098895 avg.loss:  2.244689 ETA:   0h31m31s

In [ ]:
model_label_3 = MyFastText()
model_label_3.train(
    input = TEMP_DIR + 'set3.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)
model_label_3.save(config['model_path'] + config['model_label_2_name'])

In [9]:
try:
    os.remove(TEMP_DIR + 'set1.txt')
    os.remove(TEMP_DIR + 'set2.txt')
    os.remove(TEMP_DIR + 'set3.txt')
    os.removedirs(TEMP_DIR)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: './tmp/set1.txt'


In [2]:
model_label1 = MyFastText()
model_label1.load_model('./model/model_for_label_1.model')
model_label2 = MyFastText()
model_label2.load_model('./model/model_for_label_2.model')
model_label3 = MyFastText()
model_label3.load_model('./model/model_for_label_3.model')

In [3]:
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_lable_1 = 0
true_lable_2 = 0
true_lable_3 = 0

# label_1_dict = {}
# label_2_dict = {}
# label_3_dict = {}
for id in content.keys():
    text = content[id]['text']
    
    label_1 = content[id]['label_level_1']
    label_2 = resetLabelLv2(content[id]['label_level_2'])
    label_3 = resetLabelLv3(content[id]['label_level_3'])

    # if label_1 in label_1_dict:
    #     label_1_dict[label_1][0] += 1
    # else:
    #     label_1_dict[label_1] = [1, 0]

    # if label_2 in label_2_dict:
    #     label_2_dict[label_2][0] += 1
    # else:
    #     label_2_dict[label_2] = [1, 0]
    
    
    predict_1 = model_label1.predict(text)[0][0]
    predict_2 = model_label2.predict(text)[0][0]
    predict_3 = model_label3.predict(text)[0][0]

    if(predict_1.replace('__label__', '') == label_1):
        true_lable_1 += 1
        # label_1_dict[label_1][1] += 1
    if(predict_2.replace('__label__', '') == label_2):
        true_lable_2 += 1
        # label_2_dict[label_2][1] += 1
    if(predict_3.replace('__label__', '') == label_3):
        true_lable_3 += 1
    total += 1
print('label level 1 accurate: {}% ({}/{})'.format(true_lable_1 * 100 / total, true_lable_1, total))
print('tag level 2 accurate: {}% ({}/{})'.format(true_lable_2 * 100 / total, true_lable_2, total))
print('tag level 3 accurate: {}% ({}/{})'.format(true_lable_3 * 100 / total, true_lable_3, total))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.743 seconds.
Prefix dict has been built successfully.


label level 1 accurate: 91.56013566397078% (35095/38330)
tag level 2 accurate: 70.33133315940516% (26958/38330)
tag level 3 accurate: 60.0% (22998/38330)


In [4]:
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_lable_1 = 0
true_lable_2 = 0
true_lable_3 = 0
for id in content.keys():
    text = content[id]['text']
    
    label_1 = content[id]['label_level_1']
    label_2 = resetLabelLv2(content[id]['label_level_2'])
    label_3 = resetLabelLv3(content[id]['label_level_3'])
    
    
    predict_1 = model_label1.split_and_predict(text)[0][0]
    predict_2 = model_label2.split_and_predict(text)[0][0]
    predict_3 = model_label3.split_and_predict(text)[0][0]

    if(predict_1.replace('__label__', '') == label_1):
        true_lable_1 += 1
    if(predict_2.replace('__label__', '') == label_2):
        true_lable_2 += 1
    if(predict_3.replace('__label__', '') == label_3):
        true_lable_3 += 1
    total += 1
print('label level 1 accurate: {}% ({}/{})'.format(true_lable_1 * 100 / total, true_lable_1, total))
print('tag level 2 accurate: {}% ({}/{})'.format(true_lable_2 * 100 / total, true_lable_2, total))
print('tag level 3 accurate: {}% ({}/{})'.format(true_lable_3 * 100 / total, true_lable_3, total))

label level 1 accurate: 92.06626663188104% (35289/38330)
tag level 2 accurate: 66.3866423167232% (25446/38330)
tag level 3 accurate: 52.783720323506394% (20232/38330)


lr = 0.1 epoch = 25

标签数量